In [241]:
import numpy as np 
import pandas as pd
import csv 
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler 
from imblearn.under_sampling import RandomUnderSampler 
from sklearn.tree import DecisionTreeClassifier
import os 

os.chdir("/Users/susanchen/Documents/GitHub/skincare_classification/Plots")
from collections import Counter 

## Building Multi-label Classifiers 
### Base Classifiers to test: Binary Relevance (no correlation), Binary Relevance (with correlation), Multi-label Random Forest, Multi-label Decision Tree, Classifier Chain, Binary Releveance with Stacking Aggregation, Cross-Coupling Aggregation (to combat class-imbalances)

In [242]:
data = pd.read_csv('/Users/susanchen/Documents/GitHub/skincare_classification/Data_sets/Master_cleaned.csv')

In [243]:
smallClasses = ["['Oily', 'Sensitive']", "['Dry', 'Normal', 'Oily']", "['Dry', 'Oily']", "['Dry', 'Normal', 'Oily', 'Sensitive']"]
data = data[data.Skin_Type != "['Oily', 'Sensitive']"]
data = data[data.Skin_Type != "['Dry', 'Normal', 'Oily']"]
data = data[data.Skin_Type != "['Dry', 'Oily']"]
data = data[data.Skin_Type != "['Dry', 'Normal', 'Oily', 'Sensitive']"]
data

,Product,Brand,Ingredients,Price,Skin_Type,active_ingredient,inactive_ingredient,active_ingredient_list,inactive_ingredient_list,Is_alphabetical,...,num_of_Emollients,num_of_Hydration,num_of_Skin-Restoring,num_of_Plant Extracts,num_of_Preservatives,num_of_Skin-Softening,num_of_Sensitizing,num_of_Skin-Replenishing,top_3,top3_category_list
0,#InstantDetox Facial Mask,Biobelle,"Water, Butylene Glycol, Glycerin, Trehalose, H...",3.99,['Oily'],NaN,"Water, Butylene Glycol, Glycerin, Trehalose, H...",[],"['Water', 'Butylene Glycol', 'Glycerin', 'Treh...",False,...,0,0,0,0,0,0,0,1,"['Butylene Glycol', 'Glycerin', 'Trehalose']","['Texture Enhancer', 'Skin-Replenishing, Skin-..."
1,#Peachy Facial Mask,Biobelle,"Water, Methylpropanediol, Butylene Glycol, Gly...",3.99,['Dry'],NaN,"Water, Methylpropanediol, Butylene Glycol, Gly...",[],"['Water', 'Methylpropanediol', 'Butylene Glyco...",False,...,0,0,0,0,0,0,0,1,"['Methylpropanediol', 'Butylene Glycol', 'Glyc...","[None, 'Texture Enhancer', 'Skin-Replenishing,..."
2,#Rise&Shine Facial Mask,Biobelle,"Water, Glycerin, Butylene Glycol, Triethylhexa...",3.99,['Combination'],NaN,"Water, Glycerin, Butylene Glycol, Triethylhexa...",[],"['Water', 'Glycerin', 'Butylene Glycol', 'Trie...",False,...,0,0,0,0,0,0,0,1,"['Glycerin', 'Butylene Glycol', 'Triethylhexan...","['Skin-Replenishing, Skin-Restoring', 'Texture..."
3,#RoséAllDay Facial Mask,Biobelle,"Water, Methylpropanediol, Glycerin, Propanedio...",3.99,['Combination'],NaN,"Water, Methylpropanediol, Glycerin, Propanedio...",[],"['Water', 'Methylpropanediol', 'Glycerin', 'Pr...",False,...,0,0,0,1,2,0,0,2,"['Methylpropanediol', 'Glycerin', 'Propanediol']","[None, 'Skin-Replenishing, Skin-Restoring', None]"
4,#VitaminSea Facial Mask,Biobelle,"Water, Butylene Glycol, Glycerin, Hydroxyaceto...",3.99,['Dry'],NaN,"Water, Butylene Glycol, Glycerin, Hydroxyaceto...",[],"['Water', 'Butylene Glycol', 'Glycerin', 'Hydr...",False,...,0,0,0,1,0,0,0,1,"['Butylene Glycol', 'Glycerin', 'Hydroxyacetop...","['Texture Enhancer', 'Skin-Replenishing, Skin-..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,Youthful Vitamin C Fresh Radiance Essence,No7,"Aqua (Water), Butylene Glycol, Glycerin, Gluco...",24.99,"['Combination', 'Dry', 'Normal', 'Oily']",NaN,"Aqua (Water), Butylene Glycol, Glycerin, Gluco...",[],"['Aqua (Water)', 'Butylene Glycol', 'Glycerin'...",False,...,0,0,0,2,3,0,0,2,"['Butylene Glycol', 'Glycerin', 'Gluconolactone']","['Texture Enhancer', 'Skin-Replenishing, Skin-..."
2020,Yuza Sorbet Day Cream,Erborian,"Aqua/Water, Cyclomethicone, Glycerin, Nylon-12...",48.00,"['Combination', 'Dry', 'Normal', 'Oily', 'Sens...",NaN,"Aqua/Water, Cyclomethicone, Glycerin, Nylon-12...",[],"['Aqua/Water', 'Cyclomethicone', 'Glycerin', '...",False,...,1,0,0,1,1,0,0,1,"['Cyclomethicone', 'Glycerin', 'Nylon-12']","['Emollients', 'Skin-Replenishing, Skin-Restor..."
2021,Yuza Sorbet Night Treatment,Erborian,"Aqua/Water, Cyclomethicone, Glycerin, Cetearyl...",55.00,"['Combination', 'Dry', 'Normal', 'Oily', 'Sens...",NaN,"Aqua/Water, Cyclomethicone, Glycerin, Cetearyl...",[],"['Aqua/Water', 'Cyclomethicone', 'Glycerin', '...",False,...,2,0,0,0,0,0,0,1,"['Cyclomethicone', 'Glycerin', 'Cetearyl Alcoh...","['Emollients', 'Skin-Replenishing, Skin-Restor..."
2022,Yuzu Overnight Moisture Mask,Earth Therapeutics,"Water (Aqua), Propanediol, Glycerin, Hydrogena...",7.00,"['Combination', 'Dry', 'Normal', 'Sensitive']",NaN,"Water (Aqua), Propanediol, Glycerin, Hydrogena...",[],"['Water (Aqua)', 'Propanediol', 'Glycerin', 'H...",False,...,2,0,0,1,0,0,0,1,"['Propanediol', 'Glycerin', 'Hydrogenated Poly...","[None, 'Skin-Replenishing, Skin-Restoring', None]"


In [244]:
X = data[data.columns[12:23]].values
y = data.Skin_Type
#['Oily', 'Sensitive']": 2, "['Dry', 'Normal', 'Oily']": 2, "['Dry', 'Oily']": 1, "['Dry', 'Normal', 'Oily', 'Sensitive']": 1
#summarize class distribution 
counter = Counter(y)
print(counter)



Counter({"['Combination', 'Dry', 'Normal', 'Oily', 'Sensitive']": 918, "['Combination', 'Dry', 'Normal', 'Oily']": 239, "['Combination', 'Normal', 'Oily']": 113, "['Combination', 'Oily']": 91, "['Normal']": 83, "['Dry', 'Normal']": 72, "['Dry']": 71, "['Combination', 'Dry', 'Normal']": 66, "['Combination', 'Dry', 'Normal', 'Sensitive']": 61, "['Combination']": 59, "['Sensitive']": 46, "['Combination', 'Normal']": 30, "['Dry', 'Normal', 'Sensitive']": 29, "['Oily']": 26, "['Combination', 'Normal', 'Oily', 'Sensitive']": 23, "['Combination', 'Dry']": 22, "['Dry', 'Sensitive']": 19, "['Combination', 'Dry', 'Oily']": 13, "['Normal', 'Sensitive']": 12, "['Combination', 'Oily', 'Sensitive']": 10, "['Combination', 'Dry', 'Sensitive']": 7, "['Combination', 'Dry', 'Oily', 'Sensitive']": 4, "['Normal', 'Oily']": 4})


In [245]:
#SMOTE
# transform the dataset
oversample = SMOTE(k_neighbors=3, sampling_strategy = {"['Combination', 'Normal', 'Oily']": 300, "['Combination', 'Oily']": 300, "['Normal']": 300, "['Dry', 'Normal']": 300, "['Dry']": 300, "['Combination', 'Dry', 'Normal']": 300, "['Combination', 'Dry', 'Normal', 'Sensitive']": 300, "['Combination']": 300, "['Sensitive']": 300, "['Combination', 'Normal']": 300, "['Dry', 'Normal', 'Sensitive']": 300, "['Oily']": 300, "['Combination', 'Normal', 'Oily', 'Sensitive']": 300, "['Combination', 'Dry']": 300, "['Dry', 'Sensitive']": 300, "['Combination', 'Dry', 'Oily']": 300, "['Normal', 'Sensitive']": 300, "['Combination', 'Oily', 'Sensitive']": 300, "['Combination', 'Dry', 'Sensitive']": 300, "['Combination', 'Dry', 'Oily', 'Sensitive']": 300, "['Normal', 'Oily']": 300})
X, y = oversample.fit_resample(X, y)
# summarize the new class distribution
counter = Counter(y)
print(counter)

under = RandomUnderSampler(sampling_strategy= {"['Combination', 'Dry', 'Normal', 'Oily', 'Sensitive']": 800})
X, y = under.fit_resample(X, y)


counter = Counter(y)
print(counter)

Counter({"['Combination', 'Dry', 'Normal', 'Oily', 'Sensitive']": 918, "['Oily']": 300, "['Dry']": 300, "['Combination']": 300, "['Dry', 'Sensitive']": 300, "['Sensitive']": 300, "['Combination', 'Oily']": 300, "['Combination', 'Normal', 'Oily']": 300, "['Dry', 'Normal']": 300, "['Combination', 'Dry']": 300, "['Combination', 'Dry', 'Oily']": 300, "['Normal']": 300, "['Combination', 'Dry', 'Normal', 'Sensitive']": 300, "['Combination', 'Dry', 'Oily', 'Sensitive']": 300, "['Combination', 'Oily', 'Sensitive']": 300, "['Combination', 'Dry', 'Sensitive']": 300, "['Combination', 'Dry', 'Normal']": 300, "['Dry', 'Normal', 'Sensitive']": 300, "['Combination', 'Normal']": 300, "['Combination', 'Normal', 'Oily', 'Sensitive']": 300, "['Normal', 'Sensitive']": 300, "['Normal', 'Oily']": 300, "['Combination', 'Dry', 'Normal', 'Oily']": 239})
Counter({"['Combination', 'Dry', 'Normal', 'Oily', 'Sensitive']": 800, "['Combination', 'Dry', 'Normal', 'Sensitive']": 300, "['Combination', 'Dry', 'Normal']"

In [117]:
# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy="minority", random_state=6354)
X_over, y_over = oversample.fit_resample(X, y)
# define undersample strategy
undersample = RandomUnderSampler(sampling_strategy='majority')
X_under, y_under = oversample.fit_resample(X_over, y_over)
# summarize the new class distribution
counter = Counter(y)
print(counter)

Counter({"['Combination', 'Dry', 'Normal', 'Oily', 'Sensitive']": 918, "['Combination', 'Dry', 'Normal', 'Oily']": 239, "['Combination', 'Normal', 'Oily']": 113, "['Combination', 'Oily']": 91, "['Normal']": 83, "['Dry', 'Normal']": 72, "['Dry']": 71, "['Combination', 'Dry', 'Normal']": 66, "['Combination', 'Dry', 'Normal', 'Sensitive']": 61, "['Combination']": 59, "['Sensitive']": 46, "['Combination', 'Normal']": 30, "['Dry', 'Normal', 'Sensitive']": 29, "['Oily']": 26, "['Combination', 'Normal', 'Oily', 'Sensitive']": 23, "['Combination', 'Dry']": 22, "['Dry', 'Sensitive']": 19, "['Combination', 'Dry', 'Oily']": 13, "['Normal', 'Sensitive']": 12, "['Combination', 'Oily', 'Sensitive']": 10, "['Combination', 'Dry', 'Sensitive']": 7, "['Combination', 'Dry', 'Oily', 'Sensitive']": 4, "['Normal', 'Oily']": 4, "['Oily', 'Sensitive']": 2, "['Dry', 'Normal', 'Oily']": 2, "['Dry', 'Oily']": 1, "['Dry', 'Normal', 'Oily', 'Sensitive']": 1})


In [246]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

#X = data[data.columns[12:23]].values 
#y = data.Skin_Type
y = MultiLabelBinarizer().fit_transform(y)

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, shuffle=True, test_size=0.4, random_state = 1024)
X_val, X_test, y_val, y_test = train_test_split(X_holdout, y_holdout, shuffle=False, test_size=0.5, random_state = 1024)

print("Dataset sizes:\n\tTrain %s\n\tVal %s\n\tTest %s" % (X_train.shape, X_val.shape,X_test.shape))

Dataset sizes:
	Train (4403, 11)
	Val (1468, 11)
	Test (1468, 11)


## Binary Relevance with Gaussian 

In [236]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
import sklearn.metrics as metrics
# initialize binary relevance multi-label classifier with a gaussian naive bayes base classifier
classifier = BinaryRelevance(
    classifier = GaussianNB(),
    require_dense=[True, True] 
)
# train
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_val)
# accuracy
print("Accuracy = ", metrics.accuracy_score(y_val,predictions))
print("F1 Score = ", metrics.f1_score(y_val,predictions, average = "micro"))
print("Hamming Loss", metrics.hamming_loss(y_val, predictions))


Accuracy =  0.03337874659400545
F1 Score =  0.6904925652761639
Hamming Loss 0.3865359554555147


## One vs the Rest Classifier 

In [237]:
from sklearn.multiclass import OneVsRestClassifier
classif = OneVsRestClassifier(SVC(kernel='linear', random_state= 1024))
classif.fit(X_train, y_train)
val_pred = classif.predict(X_val)
# accuracy
print("Accuracy = ", metrics.accuracy_score(y_val, val_pred))
print("F1 Score = ", metrics.f1_score(y_val,val_pred, average = "micro"))
print("Hamming Loss", metrics.hamming_loss(y_val, val_pred))

Accuracy =  0.03201634877384196
F1 Score =  0.8391369447537144
Hamming Loss 0.24819334202108756


## Decision Tree Classifier

In [238]:
from sklearn.model_selection import GridSearchCV
dt =  DecisionTreeClassifier()
param_grid = {
    'max_depth' :[5,6,7,8,9],
    'max_leaf_nodes': [5,10,15]
}
gs = GridSearchCV(dt, param_grid, scoring = "accuracy", cv= 5)

gs.fit(X, y)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [5, 6, 7, 8, 9],
                         'max_leaf_nodes': [5, 10, 15]},
             scoring='accuracy')

In [239]:
tree = dt.fit(X_train, y_train)
y_pred = tree.predict(X_test)
print(metrics.f1_score(y_test, y_pred, average="micro"))

0.8974625623960066


  ## Binary Relevance with Random Forest Classifier 

In [200]:
from sklearn.ensemble import RandomForestClassifier
import time 
start=time.time()
classifier = BinaryRelevance(
    classifier = RandomForestClassifier(),
    require_dense = [False, True]
)

classifier.fit(X_train, y_train)

print('training time taken: ',round(time.time()-start,0),'seconds')


training time taken:  28.0 seconds


In [240]:
y_hat=classifier.predict(X_test)
a=metrics.accuracy_score(y_test, y_hat)
br_f1=metrics.f1_score(y_test, y_hat, average='micro')
br_hamm=metrics.hamming_loss(y_test,y_hat)

print('Binary Relevance accuracy:',round(a,3))
print('Binary Relevance F1-score:',round(br_f1,3))
print('Binary Relevance Hamming Loss:',round(br_hamm,3))

Binary Relevance accuracy: 0.046
Binary Relevance F1-score: 0.692
Binary Relevance Hamming Loss: 0.379


## Label Powerset with Random Forest Classifier

In [204]:
from skmultilearn.problem_transform import LabelPowerset

classifier = LabelPowerset(
    classifier = RandomForestClassifier(),
    require_dense = [False, True]
)

start=time.time()
classifier.fit(X_train, y_train)
print('training time taken: ',round(time.time()-start,0),'seconds')

training time taken:  2.0 seconds


In [250]:
y_hat=classifier.predict(X_test)
lp_a=metrics.accuracy_score(y_test, y_hat)
lp_f1=metrics.f1_score(y_test, y_hat, average='micro')
lp_hamm=metrics.hamming_loss(y_test,y_hat)
print('Binary Relevance accuracy:',round(lp_a,3))
print('Label Powerset F1-score:',round(lp_f1,3))
print('Label Powerset Hamming Loss:',round(lp_hamm,3))

Binary Relevance accuracy: 0.04
Label Powerset F1-score: 0.69
Label Powerset Hamming Loss: 0.383


## Multi-label K Nearest Neighbors

In [221]:
from skmultilearn.adapt import MLkNN
import time

parameters = {'k': range(1,3), 
              's': [0.5, 0.7, 1.0]}

score = 'f1_micro'

start=time.time()

classifier = GridSearchCV(MLkNN(), parameters, scoring=score)
classifier.fit(X_train, y_train)

print('training time taken: ',round(time.time()-start,0),'seconds')
print('best parameters :', classifier.best_params_, 'best score: ',
      classifier.best_score_)

training time taken:  132.0 seconds
best parameters : {'k': 2, 's': 0.5} best score:  0.9003037410622218


## 

## Binary Relevance K-Nearest Neighbors

In [222]:
from skmultilearn.adapt import BRkNNaClassifier

parameters = {'k': range(3,5)}
score = 'f1_samples'

start=time.time()

classifier = GridSearchCV(BRkNNaClassifier(), parameters, scoring=score)
classifier.fit(X_train, y_train)

print('training time taken: ',round(time.time()-start,0),'seconds')
print('best parameters :', classifier.best_params_,
      'best score: ',classifier.best_score_)


training time taken:  3.0 seconds
best parameters : {'k': 3} best score:  0.8856294650874726


In [226]:
BrKnn = BRkNNaClassifier(k= 3)
BrKnn.fit(X_train, y_train)
pred = BrKnn.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

0.5674386920980926


In [ ]:
import math

import pandas as pd
import numpy as np
import scipy.io as sio

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

from sklearn.cluster import KMeans
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split

In [ ]:
ratio= 0.1

def mLIFT(train_data,train_target,test_data,test_target,ratio):
    num_train, dim= train_data.shape
    num_test, num_class= test_target.shape
    
    P_Centers= []
    N_Centers= []
    
    ##### KMeans, and save the centers
    for i in range(num_class):
        print ("Performing clustering:%d/%d" %(i+ 1, num_class))
    
        p_data= train_data[train_target[:,i]== 1]
        n_data= train_data[train_target[:,i]== -1]
    
        k1= int(min( math.ceil(p_data.shape[0]* ratio), math.ceil(n_data.shape[0]* ratio) ))
        #print("k1= k2= %d" %k1)
        k2= k1;
    
        if(k1== 0):
            POS_C=[]
            zero_kmeans= KMeans(n_clusters= min(50, num_train)).fit(train_data)
            NEG_C= zero_kmeans.cluster_centers_
        else:
            # Positive
            if(p_data.shape[0]== 1):
                POS_C= p_data
            else:
                p_kmeans= KMeans(n_clusters= k1).fit(p_data)
                POS_C= p_kmeans.cluster_centers_
            # Negative
            if(n_data.shape[0]== 1):
                NEG_C= n_data
            else:
                n_kmeans= KMeans(n_clusters= k2).fit(n_data)
                NEG_C= n_kmeans.cluster_centers_
                
        # Save the cluster centers
        P_Centers.append(POS_C)
        N_Centers.append(NEG_C)
            
    #print("The size of P_Canters is %d\n" %len(P_Centers))
    
    ##### Do the map and save the models
    Models= []
    for i in range(num_class):
        print ("Building classifiers: :%d/%d" %(i+ 1, num_class))
        centers= np.vstack((P_Centers[i], N_Centers[i]))
        num_center= centers.shape[0]
        # print(num_center)
        data= []
    
        if(num_center>= 5000):
            print("Too many cluster center!")
            break
        else:
            blocksize= 5000- num_center
            num_block= int(math.ceil(num_train/ blocksize))
            # print(num_block)
            
            mFirst= True
            for j in range(num_block- 1):
                print(j)
                low= j* blocksize
                high= (j+ 1)* blocksize
                # Calculate the distance
                for k in range(num_center):
                    diff= train_data[low:high, :]- centers[k]
                    Eu_diff= np.linalg.norm(diff, axis=1)
                    if(mFirst== True):
                        mFirst= False
                        data_temp= Eu_diff
                    else:
                        data_temp= np.vstack((data_temp, Eu_diff))
                    
            
            low= (num_block- 1)* blocksize
            high= num_train
            
            # Calculate the distance
            for j in range(num_center):
                diff= train_data[low:high,:]- centers[j]
                Eu_diff= np.linalg.norm(diff, axis=1)
                if(mFirst== True):
                    mFirst= False
                    data_temp= Eu_diff
                else:
                    data_temp= np.vstack((data_temp, Eu_diff))
            
            data= data_temp.T
        
        training_instance_matrix= data
        training_label_vector= train_target[:,i]
    
        model_this= SVC(C= 10, probability=True).fit(training_instance_matrix, training_label_vector)
        #model_this= LogisticRegression(C= 0.03).fit(training_instance_matrix, training_label_vector)
        #model_this= DecisionTreeClassifier().fit(training_instance_matrix, training_label_vector)
        #model_this = AdaBoostClassifier(DecisionTreeClassifier(),
                         #algorithm="SAMME",
                         #n_estimators=50, learning_rate=0.8).fit(training_instance_matrix, training_label_vector)
        Models.append(model_this)
    
    ##### Predict
    for i in range(num_class):
        print ("Predicting: :%d/%d" %(i+ 1, num_class))
        centers= np.vstack((P_Centers[i], N_Centers[i]))
        num_center= centers.shape[0]
        # print(num_center)
        data= []
    
        if(num_center>= 5000):
            print("Too many cluster center!")
            break
        else:
            blocksize= 5000- num_center
            num_block= int(math.ceil(num_test/ blocksize))
            # print(num_block)
            
            mFirst= True
            for j in range(num_block- 1):
                print(j)
                low= j* blocksize
                high= (j+ 1)* blocksize
                # Calculate the distance
                for k in range(num_center):
                    diff= test_data[low:high, :]- centers[k]
                    Eu_diff= np.linalg.norm(diff, axis=1)
                    if(mFirst== True):
                        mFirst= False
                        data_temp= Eu_diff
                    else:
                        data_temp= np.vstack((data_temp, Eu_diff))
                    
            
            low= (num_block- 1)* blocksize
            high= num_train
            
            # Calculate the distance
            for j in range(num_center):
                diff= test_data[low:high,:]- centers[j]
                Eu_diff= np.linalg.norm(diff, axis=1)
                if(mFirst== True):
                    mFirst= False
                    data_temp= Eu_diff
                else:
                    data_temp= np.vstack((data_temp, Eu_diff))
            
            data= data_temp.T
            # print(data.shape)
    
        testing_instance_matrix= data;
        testing_label_vector= test_target[:, i]
    
        predicted_label= Models[i].predict(testing_instance_matrix)
        
        #print (predicted_label)

        print("The accuracy is: %f" %accuracy_score(testing_label_vector, predicted_label))
        #print(roc_auc_score(testing_label_vector, predicted_label))
    
    return 1


mLIFT(train_data,train_target,test_data,test_target,ratio);